# Prepare data

In [ ]:
# Uncompress the data, and restore their original structure 
# When you exectue this script, it will create the following structure 
# data
#  - merged_xyz_radiometric_Clusters_Annotations
#    - *.txt  // Each text file contain XYZ+Radiometric+Cluster+Annotations
#    - original
#      - *.txt // Each txt file contain the xyz+radiometric+annotations
#    - cluster 
#      - *.txt // Each txt file contain the xyz+annotations+clusters 
#    - annotations  
#      - *.txt // Each txt file contain the XYZ+annotations 
#  - model_RF-field_fpfh
#    - model_all.sav
#    - model_fold_x.sav where x is {1,2,3,4,5}
#    - learning.log
#  - model_RF-field_rad_fpfh
#    - model_fold_x.sav where x is {1,2}
#    - learning.log 
#  - randlanet_field_and_fieldOnlyXYZ
#    - model_RandLA-Net_field
#      - snapshots
#        - snap-XXXX 
#    - model_RandLA-Net_field_only_xyz
#      - snapshots
#        - snap-XXXX 
!python restoreData.py --action=uncompress --path2tar=../data.tar.xz

## Prepare for RandLA-NET

In [ ]:
import os 
from restoreData import dataSet
# Path to the previous generated files 
p_path2data_o = os.path.join("../data/merged_xyz_radiometric_Clusters_Annotations/original/")
p_path2data_a = os.path.join("../data/merged_xyz_radiometric_Clusters_Annotations/annotations/")
# Output path for the prepared data to randlanet 
p2rnet_out_o = os.path.join(p_path2data_o, "data2rnet")
p2rnet_out_a = os.path.join(p_path2data_a, "data2rnet")
if(not os.path.isdir(p2rnet_out_o)):
    os.mkdir(p2rnet_out_o)
if(not os.path.isdir(p2rnet_out_a)):
    os.mkdir(p2rnet_out_a)
# Set them in the required format 
print("-> Original: XYZ+Radiometric+Annotations")
dataSet(p_path2data_o, p2rnet_out_o, "rnet", verbose=True, protocol="field")
print("-> Only XYZ: XYZ+Annotations")
dataSet(p_path2data_a, p2rnet_out_a, "rnet", verbose=True, protocol="field_only_xyz")

## Prepare data for Random Forest 

In [ ]:
import os 
from restoreData import dataSet
# Path to the previous generated files 
p_path2data_o = os.path.join("../data/merged_xyz_radiometric_Clusters_Annotations/original/")
p_path2data_a = os.path.join("../data/merged_xyz_radiometric_Clusters_Annotations/annotations/")
# Output path for the prepared data to randlanet 
p2rf_out_o = os.path.join(p_path2data_o, "data2rf")
p2rf_out_a = os.path.join(p_path2data_a, "data2rf")
if(not os.path.isdir(p2rf_out_o)):
    os.mkdir(p2rf_out_o)
if(not os.path.isdir(p2rf_out_a)):
    os.mkdir(p2rf_out_a)
# Set them in the required format 
print("-> Original: XYZ+Radiometric+Annotations")
dataSet(p_path2data_o, p2rf_out_o, "rdf", verbose=True)
print("-> Only XYZ: XYZ+Annotations")
dataSet(p_path2data_a, p2rf_out_a, "rdf", verbose=True)

# Predictions

## RandLA-NET

In [ ]:
import sys 
import os
from randlanet.main_apple_tree import *

In [ ]:
path2model_rnet= "../data/randlanet_field_and_fieldOnlyXYZ/model_RandLA-Net_field_only_xyz/snapshots/snap-13001" # Trained model
path2data = "../data/merged_xyz_radiometric_Clusters_Annotations/annotations/data2rnet" # Data to randlanet 

In [ ]:
print("-> Input path: %s" %("Not found" if not os.path.isdir(path2data) else "OK" ) )
print("-> Model path: %s" %("Not found" if not os.path.isfile(path2model_rnet+".meta") else "OK" ) )

In [ ]:
# Arguments for the model
param = {"gpu":0, # -1 no GPU
         "model_path":path2model_rnet, 
         "path2data":path2data, 
         "path2output": "./", # This arg only works to save the training 
         "protocol":"field_only_xyz", 
         "trainFromCHK":False}  
# NOTE: Ensure that the subsampling points in the training are the same for the prediction[test] to do this verify
# the file called helper_tools.py

In [ ]:
import tensorflow as tf 

tf.reset_default_graph() # Ensure that the models is not being reused by any previous call 

randlanet_predict(param)
# When this metod is called, a folder called test is going to bre created and inside of this folder, is related
# the folder called prediction thatn contain *.labels files with the class of each point.

# NOTE: If you have a memory problem try to reduce the number of points in the subsampling [helper_tools.py] and also reduce 
# the batch size. Note that changing the number of point for the prediction could affect the results.

In [ ]:
# Merge labels 
# NOTE: RandLA-NET write the probabilities and the labels of each point cloud on different files, 
# To visualize the classification the predicted classes and the point cloud are going to be merged 
from randlanet.utils.merge_label_apple import *

In [ ]:
path2prediction = "test/Log_XXXXX/predictions/" # The name of the folder always is 
                                                # going to change with the date
path2data = os.path.join(param["path2data"],"test/")
OutputPath = os.path.join(path2data, "merged/")

merge_pointCloudAndLabels(path2data, "./test/", OutputPath)

## Random Forest 

In [ ]:
import os 
import glob
import numpy as np 
from machine_learning.RFClassifier import RFClassifier

In [ ]:
model = RFClassifier()
#
model_weights = "../data/model_RF-field_fpfh/model_fold_1.sav"
path2data = "../data/merged_xyz_radiometric_Clusters_Annotations/annotations/data2rf/test/"
#
OutputPath = os.path.join(path2data, "prediction_rf/")
if(not os.path.isdir(OutputPath)):
    os.mkdir(OutputPath)

In [ ]:
# Load the model 
model.load(model_weights)
P = 0.7
#
filedOnlyXYZ = True 
#
lst_f2rf = glob.glob(os.path.join(path2data,"*.txt"))
for idx, fname in enumerate(lst_f2rf, start=1):
    print("-> Loading[%i/%i]: %s" %( len(lst_f2rf), idx, os.path.split(fname)[1] ))
    pc2rf = np.loadtxt(fname)
    xyz   = pc2rf[:,0:3].reshape(-1,3)
    print(" -> shape: %s" %(str(pc2rf.shape)))
    # Remove XYZ and the annotations 
    if(filedOnlyXYZ):
        pc2rf = np.delete(pc2rf, [0,1,2,3], axis=1)# Delete XYZ Annotations 
        print("   -> New shape: %s" %(str(pc2rf.shape)))
    else: # the point cloud has the radiometric features XYZ+Radiometric+Annotations 
        pc2rf = np.delete(pc2rf, [0,1,2,6], axis=1) # Delete XYZ+radiom
        print("   -> New shape: %s" %(str(pc2rf.shape)))
    print(" -> Classifing points")
    predicted = model.predict(pc2rf)
    predicted = np.where(predicted>P, 1, 0)
    print("   -> OK")
    predicted = np.concatenate((xyz, predicted.reshape(-1,1)), axis=1)
    p2save = os.path.join(OutputPath, os.path.split(fname)[1])
    np.savetxt(p2save, predicted)
#predict(model, model_weights, path2data, path2output)

# Evaluate 

In [ ]:
import os 
import glob
from machine_learning.ModelClassifier import ModelClassifier

In [ ]:
isRnet = False
lst_predictions = glob.glob(os.path.join(OutputPath,"*.labels" if isRnet else "*.txt"))
mc = ModelClassifier()
for idx, i in enumerate(lst_predictions, start=1):
    print("Loading[%i/%i]: %s" %(len(lst_predictions), idx, os.path.split(i)[1]))
    p2p = os.path.join(OutputPath, os.path.split(i)[1])
    p2g = os.path.join(path2data, os.path.split(i)[1])
    if(isRnet):
        pcp_lbl = np.loadtxt(p2p)
    else:
        pcp_lbl = np.loadtxt(p2p)[:,-1]
    pcg_lbl = np.loadtxt(p2g)[:,-1 if isRnet else 3]
    mc.evaluate(pcg_lbl, pcp_lbl)

# Clustering 

In [ ]:
import os 
import glob
import numpy as np 
import sklearn.cluster
from post_processing.algorithm import clustering 

In [ ]:
files_annApples = glob.glob(os.path.join(OutputPath,"*.txt"))
path2wrt = os.path.join(OutputPath,"clusters/")

if(not os.path.join(path2wrt)):
    os.mkdir(path2wrt)

eps, minSamples = 0.1, 20 # 0.4, 20 funciona pero consume mucha memoria 

print("Found annotated files: %i" %(len(files_annApples)))

for idx, file2clustering in enumerate(files_annApples, start=1):
    _, actualFileName = os.path.split(file2clustering)
    print("-> Loading[%i/%i]: %s" %(len(files_annApples), idx, actualFileName))
    pointCloud2cluster = np.loadtxt(file2clustering)
    cluster = clustering(pointCloud2cluster, minSamples, eps)
    print(" -> The file will be written in: %s" %path2wrt)
    np.savetxt(path2wrt+actualFileName, cluster)# The cluster is in the last column of the file